In [1]:
import optuna
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer,classification_report, f1_score, recall_score, precision_score, roc_auc_score
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

optuna.logging.set_verbosity(optuna.logging.ERROR) 
import logging
import pickle

In [2]:
# df = pd.read_csv(r"C:\Users\ntquy\Downloads\train_data_attrition_scaling.csv")
df = pd.read_csv(r"C:\Users\HP\Pictures\New folder\data\processed\processed_scaling\train_data_attrition_scaling.csv")
df.head()


,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,...,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition
0,-0.953774,1,-0.040465,2,-1.013473,2,1,3,1,-1.206150,...,3,1,-0.527297,-0.613546,3,0.011597,0.772856,-0.676110,0.843327,0
1,0.241886,1,-0.878823,1,-0.154619,1,1,3,0,-0.912823,...,1,1,-0.399283,0.157319,3,0.175907,-0.334520,-0.676110,0.843327,0
2,0.024494,2,0.968031,1,-0.522699,2,3,4,1,-0.277281,...,3,2,-0.399283,-0.613546,2,-0.152713,-0.611364,-0.676110,-0.007719,0
3,-0.192899,2,0.844743,1,1.685783,4,3,3,0,-1.792805,...,3,1,-0.911340,0.157319,3,-0.809953,-0.611364,-0.034378,-0.575084,0
4,-0.084203,2,-0.723480,1,-0.768086,3,5,3,1,-1.792805,...,4,0,-0.143254,-0.613546,3,0.175907,-1.165051,1.569949,0.843327,0


In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
X = df.drop(['Attrition'], axis=1)
y = df['Attrition'] 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
def objective(trial):
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    C = trial.suggest_float('C', 1e-5, 1e2, log=True)
    if penalty == 'l1':
        solver = trial.suggest_categorical('solver_l1', ['liblinear', 'saga'])
    else:
        solver = trial.suggest_categorical('solver_l2', ['liblinear', 'saga'])
    max_iter = trial.suggest_int('max_iter', 1000, 5000)

    model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=max_iter,class_weight='balanced', random_state=42)
    f1_scorer = make_scorer(f1_score)
    roc_auc_scorer = make_scorer(roc_auc_score)
    f1_score_cv = cross_val_score(model, X_train, y_train, cv=5, scoring=f1_scorer).mean()
    roc_auc_cv = cross_val_score(model, X_train, y_train, cv=5, scoring=roc_auc_scorer).mean()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('roc_auc', roc_auc)
    return (f1_score_cv + roc_auc_cv) / 2
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print("Best hyperparameters: ", study.best_params)
print("Best combined score: ", study.best_value)

best_params = study.best_params
final_model = LogisticRegression(
    penalty=best_params['penalty'],
    C=best_params['C'],
    solver=best_params['solver_l1'] if best_params['penalty'] == 'l1' else best_params['solver_l2'],
    max_iter=best_params['max_iter'],
    random_state=42
)
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
f1_test = f1_score(y_test, y_pred)
y_pred_proba = final_model.predict_proba(X_test)[:, 1]
roc_auc_test = roc_auc_score(y_test, y_pred_proba)
print(f"Test F1-score: {f1_test:.4f}")
print(f"Test ROC-AUC: {roc_auc_test:.4f}")

best_trial = study.best_trial
print("Tham số tốt nhất cho Logistic Regression:")
print(best_trial.params)
print(f"F1 Score cross-validated tốt nhất: {best_trial.value:.4f}")
print(f"Recall trên tập test: {best_trial.user_attrs['recall']:.4f}")
print(f"Precision trên tập test: {best_trial.user_attrs['precision']:.4f}")
print(f"ROC AUC Score trên tập test: {best_trial.user_attrs['roc_auc']:.4f}")

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(
        penalty='l2', 
        C=best_params['C'],
        solver='lbfgs',
        max_iter=best_params['max_iter'],
        class_weight='balanced',
        random_state=42
    ))
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
print("\nĐánh giá cuối cùng trên tập kiểm tra:")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")
print("\nBáo cáo phân loại chi tiết:")
print(classification_report(y_test, y_pred, digits=3))


Best hyperparameters:  {'penalty': 'l1', 'C': 0.31526718004923765, 'solver_l1': 'saga', 'max_iter': 4489}
Best combined score:  0.8299659587697776
Test F1-score: 0.8418
Test ROC-AUC: 0.9030
Tham số tốt nhất cho Logistic Regression:
{'penalty': 'l1', 'C': 0.31526718004923765, 'solver_l1': 'saga', 'max_iter': 4489}
F1 Score cross-validated tốt nhất: 0.8300
Recall trên tập test: 0.8471
Precision trên tập test: 0.8365
ROC AUC Score trên tập test: 0.9030

Đánh giá cuối cùng trên tập kiểm tra:
F1 Score: 0.8354
Recall: 0.8408
Precision: 0.8302
ROC AUC Score: 0.9043

Báo cáo phân loại chi tiết:
              precision    recall  f1-score   support

           0      0.840     0.829     0.834       158
           1      0.830     0.841     0.835       157

    accuracy                          0.835       315
   macro avg      0.835     0.835     0.835       315
weighted avg      0.835     0.835     0.835       315



In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(
        penalty='l2', 
        C=best_params['C'],
        solver='lbfgs',
        max_iter=best_params['max_iter'],
        class_weight='balanced',
        random_state=42
    ))
])
pipeline.fit(X, y)
with open(r'LogisticRegression_model.pkl', 'wb') as f:
    pickle.dump(pipeline, f)
print("\nMô hình đã được lưu vào 'LogisticRegression_model.pkl'")


Mô hình đã được lưu vào 'LogisticRegression_model.pkl'
